# 02 - Data Processing & Name Standardization
**Project**: Premier League Competitiveness Analysis (2000-2024)  
**Purpose**: Standardize club names across all three tables and create a unified dataset  
**Input**: `squad_market_values`, `transfer_spending`, `league_tables`  
**Output**: `unified_season_data` — single table joining all financial and on-field data

In [0]:
# Load our Delta tables
df_squad = spark.table("squad_market_values")
df_transfers = spark.table("transfer_spending")
df_league = spark.table("league_tables")

print(f"Squad values:     {df_squad.count()} rows")
print(f"Transfer spending: {df_transfers.count()} rows")
print(f"League tables:     {df_league.count()} rows")

In [0]:
# Get unique club names from each table
squad_names = sorted([row.club for row in df_squad.select("club").distinct().collect()])
league_names = sorted([row.club for row in df_league.select("club").distinct().collect()])
transfer_names = sorted([row.club for row in df_transfers.select("club").distinct().collect()])

# Print side by side to spot differences
print(f"{'LEAGUE TABLE':<30} {'SQUAD VALUES':<35} {'TRANSFERS':<35}")
print("=" * 100)

all_names = sorted(set(league_names + squad_names + transfer_names))
for name in all_names:
    in_league = "✓" if name in league_names else ""
    in_squad = "✓" if name in squad_names else ""
    in_transfer = "✓" if name in transfer_names else ""
    print(f"{name:<30} {in_league:<2} | {name if in_squad else '':<33} {in_transfer:<2}")

In [0]:
# League table names -> Full names (used by squad values & transfers)
name_mapping = {
    "Arsenal": "Arsenal FC",
    "Birmingham": "Birmingham City",
    "Blackburn": "Blackburn Rovers",
    "Blackpool": "Blackpool FC",
    "Bolton": "Bolton Wanderers",
    "Bournemouth": "AFC Bournemouth",
    "Bradford": "Bradford City",
    "Brentford": "Brentford FC",
    "Brighton": "Brighton & Hove Albion",
    "Burnley": "Burnley FC",
    "Cardiff": "Cardiff City",
    "Charlton": "Charlton Athletic",
    "Chelsea": "Chelsea FC",
    "Coventry": "Coventry City",
    "Derby": "Derby County",
    "Everton": "Everton FC",
    "Fulham": "Fulham FC",
    "Huddersfield": "Huddersfield Town",
    "Ipswich": "Ipswich Town",
    "Leeds": "Leeds United",
    "Leicester": "Leicester City",
    "Liverpool": "Liverpool FC",
    "Luton": "Luton Town",
    "Man City": "Manchester City",
    "Man Utd": "Manchester United",
    "Middlesbrough": "Middlesbrough FC",
    "Newcastle": "Newcastle United",
    "Norwich": "Norwich City",
    "Nott'm Forest": "Nottingham Forest",
    "Portsmouth": "Portsmouth FC",
    "QPR": "Queens Park Rangers",
    "Reading": "Reading FC",
    "Sheff Utd": "Sheffield United",
    "Southampton": "Southampton FC",
    "Sunderland": "Sunderland AFC",
    "Swansea": "Swansea City",
    "Tottenham": "Tottenham Hotspur",
    "Watford": "Watford FC",
    "West Brom": "West Bromwich Albion",
    "West Ham": "West Ham United",
    "Wigan": "Wigan Athletic",
    "Wolves": "Wolverhampton Wanderers",
}

# Names that are already correct (same across all tables)
already_matching = ["Aston Villa", "Crystal Palace", "Hull City", "Stoke City"]

print(f"Names to map: {len(name_mapping)}")
print(f"Already matching: {len(already_matching)}")
print(f"Total clubs covered: {len(name_mapping) + len(already_matching)}")

In [0]:
from pyspark.sql.functions import create_map, lit, col, coalesce

# Build a Spark mapping expression
mapping_expr = create_map([item for pair in name_mapping.items() for item in [lit(pair[0]), lit(pair[1])]])

# Apply mapping: if club name is in the map, replace it; otherwise keep original
df_league_clean = df_league.withColumn(
    "club",
    coalesce(mapping_expr[col("club")], col("club"))
)

# Verify: check that all league names now match squad value names
league_clean_names = set([row.club for row in df_league_clean.select("club").distinct().collect()])
squad_names_set = set(squad_names)

mismatches = league_clean_names - squad_names_set
if mismatches:
    print(f"Still mismatched: {mismatches}")
else:
    print("All names match!")
    
# Show a sample to confirm
df_league_clean.filter(col("season") == "2023/24").select("position", "club", "points").orderBy("position").show(5, truncate=False)

## Join All Three Tables
Join league tables (on-field data) with squad values and transfer spending (financial data) on `club` + `season_start_year`. Using a LEFT join from league tables because that's our primary dataset — every club that played in a season should appear, even if financial data is missing (2000-2003).

In [0]:
df_unified = df_league_clean.join(
    df_squad.select("season_start_year", "club", "squad_size", "avg_age", 
                     "total_market_value_eur", "has_financial_data"),
    on=["season_start_year", "club"],
    how="left"
).join(
    df_transfers.select("season_start_year", "club", "expenditure_eur", 
                         "income_eur", "net_spend_eur", "arrivals", "departures"),
    on=["season_start_year", "club"],
    how="left"
)

print(f"Unified rows: {df_unified.count()}")
print(f"Columns: {len(df_unified.columns)}")
df_unified.printSchema()

In [0]:
# Check Man City 2023/24
df_unified.filter(
    (col("club") == "Manchester City") & (col("season_start_year") == 2023)
).show(1, vertical=True)

In [0]:
df_unified.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("unified_season_data")

# Verify
spark.sql("SHOW TABLES").show()

In [0]:
spark.sql("""
    SELECT season, club, position, points, 
           total_market_value_eur, expenditure_eur
    FROM unified_season_data
    WHERE season_start_year = 2000 AND position <= 3
    ORDER BY position
""").show()

## Summary
### What This Notebook Did
1. Loaded three Delta tables from Notebook 01
2. Standardized club names across all tables (46 clubs mapped)
3. Joined league tables + squad values + transfer spending into one unified dataset
4. Saved as `unified_season_data` (500 rows, 21 columns)

### Tables Available
| Table | Description |
|-------|-------------|
| `squad_market_values` | Raw squad valuations (source data) |
| `transfer_spending` | Raw transfer data (source data) |
| `league_tables` | Raw standings (source data) |
| `unified_season_data` | **Joined dataset — use this for all analysis** |

### Data Notes
- Squad market values are €0 for 2000/01 - 2003/04 (not tracked by Transfermarkt)
- Transfer spending is available for all 25 seasons
- `has_financial_data` flag indicates whether squad values exist

### Next Step
`03_metric_calculations`: Calculate Gini coefficients, competitive balance metrics, and financial inequality measures